In [1]:
import torch.utils.data as Data
import pickle
import torch
import numpy as np
import json
from VGG_16_BN_with_FM import VGG16
import os
import torchvision.transforms as T
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from collections import Counter
import torchvision.datasets as datasets
from collections import Counter
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from get_critical_units import get_critical_units
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

def get_sensUnit(data_path, unit_num_per):
    '''
    not use the parameter unit_num_per in this function
    assume that the unit num is same with the unit num which one layer in one path includes
    '''
    with open(data_path, 'rb') as fr:
        paths = pickle.load(fr)

    units = {}
    sensUnits = {}
    sta = {}
    for layer in range(13):
        units[layer] = []
        sensUnits[layer] = []
    for k in paths.keys():       
        for layer in range(13):
            units[layer].extend(paths[k][layer])
    for layer in range(13):
        unit_num = len(paths[list(paths.keys())[0]][layer])
        if layer >= 6:
            unit_num = len(paths[list(paths.keys())[0]][layer])
        if unit_num < 20:
            unit_num = 20
        
        sta[layer] = []
        c = Counter(units[layer])
        mc = c.most_common(unit_num)
        for a, b in mc:
            sensUnits[layer].append(a)
            sta[layer].append(b)
    print("sta", sta)
    print("sensUnits:")
    print(sensUnits)
    return sensUnits


def get_sensUnit2(data_path, unit_num_per=30):

    with open(data_path, 'rb') as fr:
        paths = pickle.load(fr)

    units = {}
    sensUnits = {}
    sta = {}
    for layer in range(13):
        units[layer] = []
        sensUnits[layer] = []
    for k in paths.keys():       
        for layer in range(13):
            units[layer].extend(paths[k][layer])
    for layer in range(13):

        unit_num = int(feature_size[layer] * unit_num_per / 100) + 1
        sta[layer] = []
        c = Counter(units[layer])
        mc = c.most_common(unit_num)
        for a, b in mc:
            sensUnits[layer].append(a)
            sta[layer].append(b)
    print("sta", sta)
    print("sensUnits:", sensUnits)
    
    return sensUnits

feature_size = [64, 64, 128, 128, 256, 256, 256, 512, 512, 512, 512, 512, 512]

In [2]:
class mask_VGG16(nn.Module):
    
    def __init__(self, num_classes):
        super(mask_VGG16, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)#64 32 32
        self.bn1 = nn.BatchNorm2d(64)
        self.relu1 = nn.ReLU(inplace=False)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, padding=1)#64 32 32
        self.bn2 = nn.BatchNorm2d(64)
        self.relu2 = nn.ReLU(inplace=False)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=2)#64 16 16
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)#128 16 16
        self.bn3 = nn.BatchNorm2d(128)
        self.relu3 = nn.ReLU(inplace=False)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, padding=1)#128 16 16
        self.bn4 = nn.BatchNorm2d(128)
        self.relu4 = nn.ReLU(inplace=False)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)#128 8 8
        self.conv5 = nn.Conv2d(128, 256, kernel_size=3, padding=1)#256 8 8
        self.bn5 = nn.BatchNorm2d(256)
        self.relu5 = nn.ReLU(inplace=False)
        self.conv6 = nn.Conv2d(256, 256, kernel_size=3, padding=1)#256 8 8
        self.bn6 = nn.BatchNorm2d(256)
        self.relu6 = nn.ReLU(inplace=False)
        self.conv7 = nn.Conv2d(256, 256, kernel_size=3, padding=1)#256 8 8
        self.bn7 = nn.BatchNorm2d(256)
        self.relu7 = nn.ReLU(inplace=False)
        self.pool3 = nn.MaxPool2d(kernel_size=2, stride=2)#256 4 4
        self.conv8 = nn.Conv2d(256, 512, kernel_size=3, padding=1)#512 4 4
        self.bn8 = nn.BatchNorm2d(512)
        self.relu8 = nn.ReLU(inplace=False)
        self.conv9 = nn.Conv2d(512, 512, kernel_size=3, padding=1)#512 4 4
        self.bn9 = nn.BatchNorm2d(512)
        self.relu9 = nn.ReLU(inplace=False)
        self.conv10 = nn.Conv2d(512, 512, kernel_size=3, padding=1)#512 4 4
        self.bn10 = nn.BatchNorm2d(512)
        self.relu10 = nn.ReLU(inplace=False)
        self.pool4 = nn.MaxPool2d(kernel_size=2, stride=2)#512 2 2
        self.conv11 = nn.Conv2d(512, 512, kernel_size=3, padding=1)#512 2 2
        self.bn11 = nn.BatchNorm2d(512)
        self.relu11 = nn.ReLU(inplace=False)
        self.conv12 = nn.Conv2d(512, 512, kernel_size=3, padding=1)#512 2 2
        self.bn12 = nn.BatchNorm2d(512)
        self.relu12 = nn.ReLU(inplace=False)
        self.conv13 = nn.Conv2d(512, 512, kernel_size=3, padding=1)#512 2 2
        self.bn13 = nn.BatchNorm2d(512)
        self.relu13 = nn.ReLU(inplace=False)
        self.pool5 = nn.MaxPool2d(kernel_size=2, stride=2)#512 1 1
        self.fc1 = nn.Linear(512, num_classes)
        
    def forward(self, x, layer):

        feat_conv1 = self.conv1(x)
        feat_bn1 = self.bn1(feat_conv1)
        feat_conv1_relu = self.relu1(feat_bn1)
        
        if layer >= 1:
            feat_conv1_relu[:, units[0], :, :] = clean_feature_dict["feat_conv1_relu"][:, units[0], :, :] 

        feat_conv2 = self.conv2(feat_conv1_relu)
        feat_bn2 = self.bn2(feat_conv2)
        feat_conv2_relu = self.relu2(feat_bn2)
#         print(units[1])
        if layer >= 2:
            feat_conv2_relu[:, units[1], :, :] = clean_feature_dict["feat_conv2_relu"][:, units[1], :, :] 
        feat_pool1 = self.pool1(feat_conv2_relu)

        feat_conv3 = self.conv3(feat_pool1)
        feat_bn3 = self.bn3(feat_conv3)
        feat_conv3_relu = self.relu3(feat_bn3)
        if layer >= 3:
            feat_conv3_relu[:, units[2], :, :] = clean_feature_dict["feat_conv3_relu"][:, units[2], :, :] 
        feat_conv4 = self.conv4(feat_conv3_relu)
        feat_bn4 = self.bn4(feat_conv4)
        feat_conv4_relu = self.relu4(feat_bn4)
        if layer >= 4:
            feat_conv4_relu[:, units[3], :, :] = clean_feature_dict["feat_conv4_relu"][:, units[3], :, :] 
        feat_pool2 = self.pool2(feat_conv4_relu)

        feat_conv5 = self.conv5(feat_pool2)
        feat_bn5 = self.bn5(feat_conv5)
        feat_conv5_relu = self.relu5(feat_bn5)
        if layer >= 5:
            feat_conv5_relu[:, units[4], :, :] = clean_feature_dict["feat_conv5_relu"][:, units[4], :, :] 
        feat_conv6 = self.conv6(feat_conv5_relu)
        feat_bn6 = self.bn6(feat_conv6)
        feat_conv6_relu = self.relu6(feat_bn6)
        if layer >= 6:
            feat_conv6_relu[:, units[5], :, :] = clean_feature_dict["feat_conv6_relu"][:, units[5], :, :] 
        feat_conv7 = self.conv7(feat_conv6_relu)
        feat_bn7 = self.bn7(feat_conv7)
        feat_conv7_relu = self.relu7(feat_bn7)
        if layer >= 7:
            feat_conv7_relu[:, units[6], :, :] = clean_feature_dict["feat_conv7_relu"][:, units[6], :, :] 
        feat_pool3 = self.pool3(feat_conv7_relu)

        feat_conv8 = self.conv8(feat_pool3)
        feat_bn8 = self.bn8(feat_conv8)
        feat_conv8_relu = self.relu8(feat_bn8)
        if layer >= 8:
            feat_conv8_relu[:, units[7], :, :] = clean_feature_dict["feat_conv8_relu"][:, units[7], :, :] 
        feat_conv9 = self.conv9(feat_conv8_relu)
        feat_bn9 = self.bn9(feat_conv9)
        feat_conv9_relu = self.relu9(feat_bn9)
        if layer >= 9:
            feat_conv9_relu[:, units[8], :, :] = clean_feature_dict["feat_conv9_relu"][:, units[8], :, :] 
        feat_conv10 = self.conv10(feat_conv9_relu)
        feat_bn10 = self.bn10(feat_conv10)
        feat_conv10_relu = self.relu10(feat_bn10)
        if layer >= 10:
            feat_conv10_relu[:, units[9], :, :] = clean_feature_dict["feat_conv10_relu"][:, units[9], :, :] 
        feat_pool4 = self.pool4(feat_conv10_relu)

        feat_conv11 = self.conv11(feat_pool4)
        feat_bn11 = self.bn11(feat_conv11)
        feat_conv11_relu = self.relu11(feat_bn11)
        if layer >= 11:
            feat_conv11_relu[:, units[10], :, :] = clean_feature_dict["feat_conv11_relu"][:, units[10], :, :] 
        feat_conv12 = self.conv12(feat_conv11_relu)
        feat_bn12 = self.bn12(feat_conv12)
        feat_conv12_relu = self.relu12(feat_bn12)
        if layer >= 12:
            feat_conv12_relu[:, units[11], :, :] = clean_feature_dict["feat_conv12_relu"][:, units[11], :, :] 
        feat_conv13 = self.conv13(feat_conv12_relu)
        feat_bn13 = self.bn13(feat_conv13)
        feat_conv13_relu = self.relu13(feat_bn13)
        if layer >= 13:
            feat_conv13_relu[:, units[12], :, :] = clean_feature_dict["feat_conv13_relu"][:, units[12], :, :] 
        feat_pool5 = self.pool5(feat_conv13_relu)

        feat_pool5 = feat_pool5.view(feat_pool5.size(0),-1)

        after_dropout = F.dropout(feat_pool5, p=0.3, training=self.training,inplace=False)

        feat_fc1 = self.fc1(after_dropout)
        return feat_fc1, {'feat_conv1': feat_conv1, 'feat_conv2': feat_conv2, 'feat_conv3': feat_conv3, 'feat_conv4': feat_conv4, 'feat_conv5': feat_conv5, 'feat_conv6': feat_conv6, 'feat_conv7': feat_conv7, 'feat_conv8': feat_conv8, 'feat_conv9' : feat_conv9, 'feat_conv10': feat_conv10, 'feat_conv11': feat_conv11, 'feat_conv12': feat_conv12, 'feat_conv13': feat_conv13, 'feat_conv1_relu': feat_conv1_relu, 'feat_conv2_relu': feat_conv2_relu, 'feat_conv3_relu': feat_conv3_relu, 'feat_conv4_relu': feat_conv4_relu, 'feat_conv5_relu': feat_conv5_relu, 'feat_conv6_relu': feat_conv6_relu, 'feat_conv7_relu': feat_conv7_relu, 'feat_conv8_relu': feat_conv8_relu, 'feat_conv9_relu': feat_conv9_relu, 'feat_conv10_relu': feat_conv10_relu, 'feat_conv11_relu': feat_conv11_relu, 'feat_conv12_relu': feat_conv12_relu, 'feat_conv13_relu': feat_conv13_relu, 'feat_fc1': feat_fc1}

In [36]:
model = VGG16(num_classes=10).cuda()
model_path = "/media/trs1/litl/Vanilla_VGG_16_copy.pkl"
# model_path = "/media/trs1/TheVirtuoso/Model_Sensitivity/BN_VGG/PAT_VGG_16_finetune.pkl"
checkpoint = torch.load(model_path)
model.load_state_dict(torch.load(model_path))
model.eval()

mask_model = mask_VGG16(10).cuda()
model_path = "/media/trs1/litl/Vanilla_VGG_16_copy.pkl"
checkpoint = torch.load(model_path)
mask_model.load_state_dict(checkpoint)
mask_model.eval()

transform_test = T.Compose([
            T.ToTensor()
])

dataset = datasets.CIFAR10(
        root = "/media/trs1/yuhang/cifar-10",
        train = False,
        transform = transform_test,
        download = False
)
dataloader = Data.DataLoader(dataset=dataset, batch_size=64, shuffle=False, num_workers=2, drop_last=False)


def read_data_label(data_path, label_path):
    with open(data_path, 'rb') as fr:
        test_data = pickle.load(fr)
        size = len(test_data)
    with open(label_path, 'rb') as fr:
        test_label = pickle.load(fr)
    return test_data, test_label, size

attack = "MI-FGSM"
test_data_path = "/media/trs1/TheVirtuoso/Model_Sensitivity/new_adv_examples/white-box-vgg16/Vanilla/" + attack + "/test_adv(eps_0.031).pkl"
test_label_path = "/media/trs1/TheVirtuoso/Model_Sensitivity/new_adv_examples/white-box-vgg16/Vanilla/" + attack + "/test_label.pkl"
test_data, test_label, size = read_data_label(test_data_path, test_label_path)
adv_dataset = Data.TensorDataset(test_data, test_label)
adv_dataloader = Data.DataLoader(dataset=adv_dataset, batch_size=64, shuffle=False, num_workers=2, drop_last=False)

In [37]:
import random
start_index = 0

for step, (x, y) in enumerate(dataloader): 
    x = x.cuda()
    target, clean_feature_dict = model(x)  
    break
    
unit_nums = [30]  
results = []
for unit_num in unit_nums:

    # root = "/media/trs1/litl/Vanilla_key_paths_loss_lossOriented_abs/"
    root = "/media/trs1/litl/path_cifar/Vanilla_key_paths_loss_layer_abs/"
    attack = "clean"
    data_path = root + "key_paths_" + attack + "/key_paths_unit_per20.pkl"
    units_clean = get_sensUnit2(data_path, unit_num)
    
    mean_features_specialUnits = {}
    mean_features_regularUnits = {}
    num_special = {}
    num_regular = {}
    for k in units_clean.keys():
        conv_name = "feat_conv{}_relu".format(k+1)
        mean_features_specialUnits[conv_name] = 0
        mean_features_regularUnits[conv_name] = 0
        num_special[conv_name] = 0
        num_regular[conv_name] = 0
    # c_links_clean = get_criticalLinks(data_path, link_num)


#     random_units = {}
#     for i in units_clean:
#         layer = "feat_conv" + str(i+1) + "_relu"
#         random_units[i] = []
#         if unit_num < 50:
#             while len(random_units[i]) < len(units_clean[i]):
#                 r = random.randint(0, clean_feature_dict[layer].size(1)-1)
#                 if not r in units_clean[i] and not r in random_units[i]:
#                     random_units[i].append(r)
#         else:
#             for r in range(clean_feature_dict[layer].size(1)-1):
#                 if not r in units_clean[i]:
#                     random_units[i].append(r)
#     units = random_units
    units = units_clean

    # print(random_units)
    print(units_clean)

    with torch.no_grad():

        sample_num = 0
        adv_right0 = 0
        adv_right1 = 0
        adv_right3 = 0
        adv_right5 = 0
        adv_right7 = 0
        clean_right = 0
        for step, (x, y) in enumerate(dataloader):    
            print("step:", step)
            batch_size = x.size(0)
            adv_x = adv_dataset[start_index: start_index+batch_size][0]
            start_index += x.size(0)
            x = x.cuda()
            y = y.cuda()
            adv_x = adv_x.cuda()

            target, clean_feature_dict = model(x)
            adv_target0, adv_feature_dict0 = mask_model(adv_x, 0)
            adv_target1, adv_feature_dict1 = mask_model(adv_x, 1)
            adv_target3, adv_feature_dict3 = mask_model(adv_x, 3)
            adv_target5, adv_feature_dict5 = mask_model(adv_x, 5)
            adv_target7, adv_feature_dict7 = mask_model(adv_x, 7)

            target = torch.max(target.data, 1)[1].data
            adv_target0 = torch.max(adv_target0.data, 1)[1].data
            adv_target1 = torch.max(adv_target1.data, 1)[1].data
            adv_target3 = torch.max(adv_target3.data, 1)[1].data
            adv_target5 = torch.max(adv_target5.data, 1)[1].data
            adv_target7 = torch.max(adv_target7.data, 1)[1].data

            clean_right += (target == y).sum()
            adv_right0 += (adv_target0 == y).sum()
            adv_right1 += (adv_target1 == y).sum() 
            adv_right3 += (adv_target3 == y).sum() 
            adv_right5 += (adv_target5 == y).sum() 
            adv_right7 += (adv_target7 == y).sum() 

    print([adv_right0.item(), adv_right1.item(), adv_right3.item(), adv_right5.item(), adv_right7.item()])


sta {0: [8919, 8678, 8579, 8492, 8422, 8300, 8292, 7597, 7555, 7554, 7157, 7099, 6697, 6359, 6247, 6169, 5491, 4687, 4538, 4350], 1: [8545, 8511, 8326, 8226, 8075, 8048, 7993, 7826, 7676, 7246, 7191, 6226, 6157, 5827, 5622, 5209, 5145, 5040, 4760, 4468], 2: [8683, 8100, 7917, 7651, 7340, 7306, 6862, 6792, 6758, 6210, 6113, 5717, 5462, 5261, 5226, 5088, 5028, 4980, 4876, 4856, 4686, 4613, 4585, 4510, 4403, 4316, 3986, 3966, 3932, 3679, 3627, 3560, 3466, 3393, 3119, 2909, 2630, 2478, 2023], 3: [8919, 8919, 8797, 7339, 6511, 6400, 6232, 6070, 6024, 5857, 5818, 5612, 5610, 5553, 5368, 5351, 5260, 5223, 4922, 4629, 4578, 4313, 4248, 4227, 4212, 3724, 3582, 3510, 3400, 3229, 3035, 2997, 2793, 2740, 2728, 2698, 2331, 2162, 2125], 4: [8919, 8919, 8918, 8903, 8901, 8888, 8759, 8683, 8675, 8658, 8592, 8255, 8206, 8193, 8139, 8133, 8071, 7831, 7654, 7642, 7610, 7517, 7450, 7444, 7283, 7185, 7175, 7160, 7126, 7059, 6638, 6602, 6503, 6303, 6265, 6166, 6125, 5924, 5846, 5450, 5358, 5265, 5169, 5136,

{0: [3, 21, 45, 32, 2, 0, 34, 4, 46, 35, 16, 58, 26, 24, 25, 54, 30, 22, 15, 62], 1: [41, 58, 10, 7, 45, 50, 40, 51, 17, 0, 55, 43, 48, 49, 28, 26, 37, 29, 46, 30], 2: [51, 18, 90, 120, 92, 17, 26, 32, 40, 13, 89, 72, 111, 109, 23, 74, 21, 105, 124, 122, 123, 10, 83, 47, 79, 4, 59, 61, 78, 113, 50, 16, 119, 19, 126, 98, 117, 75, 104], 3: [6, 18, 49, 81, 74, 60, 103, 87, 32, 58, 110, 29, 79, 93, 105, 65, 78, 59, 26, 36, 10, 66, 42, 23, 9, 56, 97, 118, 109, 88, 4, 15, 91, 123, 121, 125, 35, 94, 54], 4: [198, 35, 241, 125, 92, 126, 16, 216, 124, 36, 78, 130, 135, 31, 235, 74, 98, 236, 66, 65, 56, 21, 129, 175, 69, 6, 166, 82, 213, 242, 190, 95, 54, 144, 46, 253, 44, 221, 63, 72, 174, 33, 121, 186, 94, 178, 87, 239, 119, 146, 37, 205, 17, 150, 41, 187, 195, 136, 148, 184, 75, 43, 142, 232, 246, 247, 32, 203, 80, 251, 194, 67, 0, 217, 248, 77, 111], 5: [163, 231, 190, 155, 138, 236, 146, 100, 74, 250, 36, 80, 187, 240, 134, 232, 19, 23, 110, 172, 64, 20, 216, 59, 27, 16, 184, 254, 15, 206, 

### 高斯噪声

In [46]:
model = VGG16(num_classes=10).cuda()
model_path = "/media/trs1/litl/Vanilla_VGG_16_copy.pkl"
# model_path = "/media/trs1/TheVirtuoso/Model_Sensitivity/BN_VGG/PAT_VGG_16_finetune.pkl"
checkpoint = torch.load(model_path)
model.load_state_dict(torch.load(model_path))
model.eval()

mask_model = mask_VGG16(10).cuda()
model_path = "/media/trs1/litl/Vanilla_VGG_16_copy.pkl"
checkpoint = torch.load(model_path)
mask_model.load_state_dict(checkpoint)
mask_model.eval()

unit_num = 20
unit_pick_path = 20


transform_test = T.Compose([
            T.ToTensor()
])

dataset = datasets.CIFAR10(
        root = "/media/trs1/yuhang/cifar-10",
        train = False,
        transform = transform_test,
        download = False
)
dataloader = Data.DataLoader(dataset=dataset, batch_size=64, shuffle=False, num_workers=2, drop_last=False)

# root = "/media/trs1/litl/Vanilla_key_paths_loss_lossOriented_abs/"
root = "/media/trs1/litl/path_cifar/Vanilla_key_paths_loss_layer_abs/"
attack = "clean"
data_path = root + "key_paths_" + attack + "/key_paths_unit_per20.pkl"
# c_links_clean = get_criticalLinks(data_path, link_num)
units_clean = get_sensUnit2(data_path, unit_num)

transform_test = T.Compose([
            T.ToTensor()
])

dataset = datasets.CIFAR10(
        root = "/media/trs1/yuhang/cifar-10",
        train = False,
        transform = transform_test,
        download = False
)

mean_features_specialUnits = {}
mean_features_regularUnits = {}
num_special = {}
num_regular = {}
for k in units_clean.keys():
    conv_name = "feat_conv{}_relu".format(k+1)
    mean_features_specialUnits[conv_name] = 0
    mean_features_regularUnits[conv_name] = 0
    num_special[conv_name] = 0
    num_regular[conv_name] = 0

data_root = '/media/trs1/yuhang/CIFAR-10-C/impulse_noise.npy'
label_root = '/media/trs1/yuhang/CIFAR-10-C/labels.npy'

def get_adv_dataset(x,y,test_batch_size):
    test_data = torch.from_numpy(x).float()
    test_label = torch.from_numpy(y).long()
    adv_dataset = torch.utils.data.TensorDataset(test_data,test_label)
    test_loader = torch.utils.data.DataLoader(dataset = adv_dataset,
                                              batch_size = test_batch_size,
                                              shuffle = False,
                                              drop_last = True)
    return adv_dataset

#load data
x = np.load(data_root)
x = x.transpose((0,3,1,2))
x = x/255.0
y = np.load(label_root)
#data_loader
adv_dataset = get_adv_dataset(x, y, 64)

sta {0: [8919, 8678, 8579, 8492, 8422, 8300, 8292, 7597, 7555, 7554, 7157, 7099, 6697], 1: [8545, 8511, 8326, 8226, 8075, 8048, 7993, 7826, 7676, 7246, 7191, 6226, 6157], 2: [8683, 8100, 7917, 7651, 7340, 7306, 6862, 6792, 6758, 6210, 6113, 5717, 5462, 5261, 5226, 5088, 5028, 4980, 4876, 4856, 4686, 4613, 4585, 4510, 4403, 4316], 3: [8919, 8919, 8797, 7339, 6511, 6400, 6232, 6070, 6024, 5857, 5818, 5612, 5610, 5553, 5368, 5351, 5260, 5223, 4922, 4629, 4578, 4313, 4248, 4227, 4212, 3724], 4: [8919, 8919, 8918, 8903, 8901, 8888, 8759, 8683, 8675, 8658, 8592, 8255, 8206, 8193, 8139, 8133, 8071, 7831, 7654, 7642, 7610, 7517, 7450, 7444, 7283, 7185, 7175, 7160, 7126, 7059, 6638, 6602, 6503, 6303, 6265, 6166, 6125, 5924, 5846, 5450, 5358, 5265, 5169, 5136, 4872, 4813, 4613, 4523, 4319, 3945, 3903, 3610], 5: [8918, 8917, 8911, 8903, 8898, 8854, 8837, 8802, 8790, 8697, 8645, 8616, 8559, 8510, 8502, 8465, 8433, 8428, 8351, 8343, 8331, 8253, 7797, 7742, 7723, 7590, 7581, 7569, 7404, 7357, 7354, 

In [47]:
import random
start_index = 0

for step, (x, y) in enumerate(dataloader): 
    x = x.cuda()
    target, clean_feature_dict = model(x)  
    break


# random_units = {}
# for i in units_clean:
#     layer = "feat_conv" + str(i+1) + "_relu"
#     random_units[i] = []
#     while len(random_units[i]) < len(units_clean[i]):
#         r = random.randint(0, clean_feature_dict[layer].size(1)-1)
#         if not r in units_clean[i] and not r in random_units[i]:
#             random_units[i].append(r)
        
# units = random_units
units = units_clean

print(random_units)
print(units_clean)

with torch.no_grad():

    sample_num = 0
    adv_right0 = 0
    adv_right1 = 0
    adv_right3 = 0
    adv_right5 = 0
    adv_right7 = 0
    clean_right = 0
    for step, (x, y) in enumerate(dataloader):    
        print("step:", step)
        batch_size = x.size(0)
        adv_x = adv_dataset[start_index: start_index+batch_size][0]
        start_index += x.size(0)
        x = x.cuda()
        y = y.cuda()
        adv_x = adv_x.cuda()

        target, clean_feature_dict = model(x)
        adv_target0, adv_feature_dict0 = mask_model(adv_x, 0)
        adv_target1, adv_feature_dict1 = mask_model(adv_x, 1)
        adv_target3, adv_feature_dict3 = mask_model(adv_x, 3)
        adv_target5, adv_feature_dict5 = mask_model(adv_x, 5)
        adv_target7, adv_feature_dict7 = mask_model(adv_x, 7)

        target = torch.max(target.data, 1)[1].data
        adv_target0 = torch.max(adv_target0.data, 1)[1].data
        adv_target1 = torch.max(adv_target1.data, 1)[1].data
        adv_target3 = torch.max(adv_target3.data, 1)[1].data
        adv_target5 = torch.max(adv_target5.data, 1)[1].data
        adv_target7 = torch.max(adv_target7.data, 1)[1].data

        clean_right += (target == y).sum()
        adv_right0 += (adv_target0 == y).sum()
        adv_right1 += (adv_target1 == y).sum() 
        adv_right3 += (adv_target3 == y).sum() 
        adv_right5 += (adv_target5 == y).sum() 
        adv_right7 += (adv_target7 == y).sum() 

print([adv_right0.item(), adv_right1.item(), adv_right3.item(), adv_right5.item(), adv_right7.item()])


{0: [54, 51, 23, 5, 24, 47, 61, 25, 40, 31, 13, 60, 50], 1: [23, 24, 35, 32, 4, 33, 52, 60, 3, 36, 8, 44, 21], 2: [101, 81, 125, 104, 37, 20, 116, 54, 96, 76, 118, 45, 102, 28, 56, 86, 60, 73, 61, 34, 59, 93, 77, 12, 53, 9], 3: [2, 119, 83, 47, 89, 54, 86, 73, 31, 19, 104, 15, 76, 57, 72, 107, 8, 70, 115, 63, 77, 27, 99, 75, 22, 111], 4: [252, 215, 225, 237, 169, 112, 164, 77, 161, 139, 11, 86, 185, 170, 246, 145, 247, 99, 217, 233, 41, 107, 140, 38, 90, 50, 101, 0, 207, 142, 202, 243, 79, 191, 148, 136, 201, 168, 22, 29, 231, 106, 68, 158, 189, 234, 91, 143, 192, 105, 165, 73], 5: [147, 207, 140, 28, 73, 93, 208, 222, 3, 66, 7, 131, 4, 85, 96, 164, 176, 119, 34, 52, 127, 81, 153, 61, 5, 170, 106, 225, 175, 14, 77, 21, 149, 55, 70, 192, 194, 144, 137, 29, 98, 188, 111, 41, 35, 174, 97, 114, 247, 223, 47, 251], 6: [124, 14, 204, 173, 201, 116, 122, 88, 199, 178, 225, 184, 180, 153, 101, 94, 235, 209, 49, 213, 241, 73, 189, 64, 47, 240, 54, 113, 205, 210, 179, 208, 70, 39, 251, 252, 58, 

step: 10
step: 11
step: 12
step: 13
step: 14
step: 15
step: 16
step: 17
step: 18
step: 19
step: 20
step: 21
step: 22
step: 23
step: 24
step: 25
step: 26
step: 27
step: 28
step: 29
step: 30
step: 31
step: 32
step: 33
step: 34
step: 35
step: 36
step: 37
step: 38
step: 39
step: 40
step: 41
step: 42
step: 43
step: 44
step: 45
step: 46
step: 47
step: 48
step: 49
step: 50
step: 51
step: 52
step: 53
step: 54
step: 55
step: 56
step: 57
step: 58
step: 59
step: 60
step: 61
step: 62
step: 63
step: 64
step: 65
step: 66
step: 67
step: 68
step: 69
step: 70
step: 71
step: 72
step: 73
step: 74
step: 75
step: 76
step: 77
step: 78
step: 79
step: 80
step: 81
step: 82
step: 83
step: 84
step: 85
step: 86
step: 87
step: 88
step: 89
step: 90
step: 91
step: 92
step: 93
step: 94
step: 95
step: 96
step: 97
step: 98
step: 99
step: 100
step: 101
step: 102
step: 103
step: 104
step: 105
step: 106
step: 107
step: 108
step: 109
step: 110
step: 111
step: 112
step: 113
step: 114
step: 115
step: 116
step: 117
step: 118
